# Agenda

1. Concurrency and parallelism in programming in general, and Python in particular
2. Basic threads
3. Joining threads
4. Switching threads and the GIL
5. Sharing data (and other resources)
6. Producer-consumer 
7. Events and timers
8. Multiprocessing
9. `concurrent.futures` and making life easier for ourselves

# Concurrency and parallelism in Python

- Concurency means: I have several things that I want to be tracking at once, even if they're not necessarily executing at once.
- Parallelism means: I have several things that I want to be tracking at once, *AND* they should also be executing at once.

If you want true parallel execution on a computer, then you need multiple cores (processors).  But you'll probably have more processes running than cores anyway, which means that the computer needs to keep track of each process, swapping it in and out of memory to the CPUs.

How can we have multiple things happen in our program, so that we can break a problem apart and deal with it using concurrency?
- The oldest, and most traditional, way is to use *processes*.  The good news is that each process runs separately, with its own memory, and is independent of other processes.  This means that the computer can decide which core runs which process, and when.  The problem is that there's a lot of overhead to that -- it takes more memory, and switching requires more time + resources.
- A newer way to do things is *threads*.  If your OS runs multiple processes, then your process can contain multiple threads. The idea is that the OS tells a process that it now has a chance to run, and then inside of that process